In [7]:
import random
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import openai
# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *
from CF_utils import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'amazon-beauty/merged_data.csv')
print(f'Data path: {data_path}')

# output
CF_OUTPUT_PATH = os.path.join(DATA_DIR, 'amazon-beauty/output/CF.csv')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'amazon-beauty/output/rerun_CF.csv')
print(f'Data path: {data_path}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/merged_data.csv
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/merged_data.csv
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/merged_data.csv


In [8]:
# Read and Merge Data
def load_and_merge_data(movies_path, ratings_path, users_path):
    # Load each file
    movies = pd.read_csv(movies_path, delimiter='::', engine= 'python', header=None, names=['asin', 'title', 'Genres'], encoding='ISO-8859-1')
    ratings = pd.read_csv(ratings_path, delimiter='::', engine= 'python', header=None, names=['reviewerID', 'asin', 'rating', 'Timestamp'], encoding='ISO-8859-1')
    users = pd.read_csv(users_path,delimiter='::', engine= 'python', header=None, names=['reviewerID', 'Gender', 'Age', 'Occupation', 'Zip-code'], encoding='ISO-8859-1')
    # Merge datasets
    merged_data = pd.merge(pd.merge(ratings, users, on='reviewerID'), movies, on='asin')
    return merged_data


# Filter Users with ≥ 5 ratings
def filter_users(data):
    user_rating_counts = data['reviewerID'].value_counts()
    valid_users = user_rating_counts[user_rating_counts >= 5].index.tolist()
    return data[data['reviewerID'].isin(valid_users)]


# Calculate Pearson Correlation Coefficient
# source RMIT courses
def pearson_correlation(interaction_matrix):
    """
    Compute the Pearson Correlation Coefficient matrix for the user-item interaction matrix.

    Args:
    interaction_matrix (csr_matrix): A sparse matrix where rows represent users and columns represent items.
                                     The values in the matrix are the ratings given by users to items.

    Returns:
    numpy.ndarray: A 2D array representing the Pearson Correlation Coefficients between each pair of users.
    """
    # Convert sparse matrix to dense format for processing
    dense_matrix = interaction_matrix.toarray()
    
    # Get the number of users
    n_users = dense_matrix.shape[0]

    # Initialize the Pearson Correlation matrix
    pearson_corr_matrix = np.zeros((n_users, n_users))

    # Small constant to avoid division by zero
    EPSILON = 1e-9

    # Iterate over each pair of users
    for i in range(n_users):
        for j in range(n_users):
            # Get the rating vectors for the current pair of users
            user_i_vec = dense_matrix[i, :]
            user_j_vec = dense_matrix[j, :]

            # Masks for rated items
            mask_i = user_i_vec > 0
            mask_j = user_j_vec > 0

            # Find indices of corrated items
            corrated_index = np.intersect1d(np.where(mask_i)[0], np.where(mask_j)[0])

            # Skip if no items are corrated
            if len(corrated_index) == 0:
                continue

            # Compute the mean rating for each user over corrated items
            mean_user_i = np.mean(user_i_vec[corrated_index])
            mean_user_j = np.mean(user_j_vec[corrated_index])

            # Compute the deviations from the mean
            user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
            user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

            # Calculate the components for Pearson correlation
            r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
            r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

            r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
            r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

            # Calculate Pearson correlation
            sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

            # Store the similarity in the matrix
            pearson_corr_matrix[i, j] = sim

    return pearson_corr_matrix


# Find Valid Neighbors
def get_valid_neighbors(pcc_matrix, threshold=0.6):
    valid_neighbors = {}
    for i, row in enumerate(pcc_matrix):
        valid_neighbors[i] = np.where(row > threshold)[0]
    return valid_neighbors

In [15]:
"""sumary_line

Keyword arguments:
argument -- description
Return: return_description

Iterates through each user in the filtered data.
Finds the top 4 similar users based on Pearson Correlation Coefficients.
Constructs the training data from these similar users' ratings.
Selects a random movie title from the main user's data for prediction.
Generates the combined text for prediction and calls the predict_rating_combined_ChatCompletion() function.
Saves the results to a CSV file.


Prints the IDs of the top 4 similar users for each main user.
Includes only a specified number of historical ratings from each similar user, as controlled by the num_ratings_per_user parameter.
Continues to save the results in a CSV file as before.

Allows dynamic setting of the number of similar users to consider for each main user via the num_similar_users parameter.
Prints out the user ID along with their associated historical ratings.
Saves the results in a CSV file.

Assume user id is string type not number
"""
def predict_ratings_with_collaborative_filtering_and_save(data, 
                                                          pcc_matrix, 
                                                          user_column_name='reviewerID', 
                                                          movie_column_name='title', 
                                                          movie_id_column='asin',
                                                          rating_column_name='rating', 
                                                          num_ratings_per_user=1, 
                                                          num_similar_users=4, 
                                                          save_path='cf_predictions.csv', 
                                                          seed=RANDOM_STATE):
    results = []

    # Create a mapping from user_id to row index in the pcc_matrix
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(data[user_column_name].unique())}

    # Random seed for reproducibility
    random.seed(seed)

    for user_id in data[user_column_name].unique():
        # Get the user's row index in the pcc_matrix
        user_idx = user_id_to_index[user_id]

        # Find the top similar users based on Pearson Correlation Coefficient
        similar_users_idx = np.argsort(-pcc_matrix[user_idx])[:num_similar_users + 1]  # Including the user itself
        similar_users_idx = similar_users_idx[similar_users_idx != user_idx][:num_similar_users]  # Exclude the user itself

        # Print the IDs of the similar users and their historical ratings
        print(f"User {user_id} - Similar Users: {[data[user_column_name].unique()[idx] for idx in similar_users_idx]}")
        similar_users_ratings = ""
        for idx in similar_users_idx:
            similar_user_id = data[user_column_name].unique()[idx]
            similar_user_data = data[data[user_column_name] == similar_user_id]
            historical_ratings = similar_user_data.head(num_ratings_per_user)
            for _, row in historical_ratings.iterrows():
                rating_info = f"* title: {row[movie_column_name]} - rating: {row[rating_column_name]} stars"
                print(f"Similar User {similar_user_id} - Historical rating: {rating_info}")
                similar_users_ratings += rating_info + "\n"

        # Select a random movie title from the main user's data
        user_data = data[data[user_column_name] == user_id]
        random_movie_row = user_data.sample(n=1, random_state=seed).iloc[0]
        random_movie_title = random_movie_row[movie_column_name]
        random_movie_id = random_movie_row[movie_id_column]
        actual_rating = random_movie_row[rating_column_name]

        # Generate the combined text for prediction
        combined_text = f"title: {random_movie_title}"

        # Get the predicted rating using ChatGPT
        predicted_rating = predict_rating_combined_ChatCompletion(
            combined_text, 
            approach="CF", 
            similar_users_ratings=similar_users_ratings
        )

        # Append the results
        results.append([user_id, random_movie_id, random_movie_title, actual_rating, predicted_rating])

    # Save results to CSV
    results_df = pd.DataFrame(results, columns=['user_id', 'item_id', 'title', 'actual_rating', 'predicted_rating'])
    results_df.to_csv(save_path, index=False)
    print(f"Predictions saved to {save_path}")
    return results_df


In [10]:
data = pd.read_csv(data_path)
data.head(3)

,rating,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price
0,5.0,True,2015-09-17,ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,2015-09-17,NaN,...,NaN,Reach,[],120.0,"['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,5.17
1,5.0,True,2015-09-17,ANV9L0JU6BNL,B000052YAN,Dennis,best floss i've used. does not break as easily...,best floss i've used,2015-09-17,NaN,...,NaN,Reach,[],120.0,"['B01I9TJRN4', 'B003XDVERE', 'B0722XHMGZ', 'B0...",{'\n Product Dimensions: \n ': '1 x 1 x ...,All Beauty,NaN,NaN,5.17
2,2.0,True,2018-03-27,A2TU781PWGS09X,B00006L9LC,Amazon Customer,Doesnt smell,Two Stars,2018-03-27,NaN,...,NaN,Citre Shine,[],1.0,[],"{'ASIN: ': 'B00006L9LC', 'UPC:': '795827187965...",All Beauty,NaN,NaN,23.00


In [11]:
# Create User-Item Interaction Matrix
interaction_matrix = pd.pivot_table(data, index='reviewerID', columns='asin', values='rating').fillna(0)
csr_interaction_matrix = csr_matrix(interaction_matrix.values)

# Calculate Pearson Correlation Coefficient Matrix
pcc_matrix = pearson_correlation(csr_interaction_matrix)

pcc_matrix

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [12]:
csr_interaction_matrix

<5x23 sparse matrix of type '<class 'numpy.float64'>'
	with 27 stored elements in Compressed Sparse Row format>

In [13]:
interaction_matrix

asin,B000052YAN,B00006L9LC,B000URXP6E,B0012Y0ZG2,B0015S8026,B001OHV1H4,B001QY8QXM,B00FE90M0K,B00L62HMQG,B00NW59IM6,...,B00VARTPKS,B010B5ZE4U,B010L79142,B017LU859E,B019DWCJDG,B019V2KYZS,B01A4XPV56,B01DWBI43I,B01FQQMNLQ,B01GZWUSYY
reviewerID,,,,,,,,,,,,,,,,,,,,,
A1U8L4X1O2LXXF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0
A2TU781PWGS09X,0.0,2.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
A38IPSLKOQ5O2F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,4.0,5.0,0.0
A3A8F2URN7MEPR,0.0,5.0,5.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ANV9L0JU6BNL,5.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [16]:
%%time

cf_predictions = predict_ratings_with_collaborative_filtering_and_save(data, pcc_matrix, save_path=CF_OUTPUT_PATH)


User ANV9L0JU6BNL - Similar Users: ['A2TU781PWGS09X', 'A3A8F2URN7MEPR', 'A1U8L4X1O2LXXF', 'A38IPSLKOQ5O2F']
Similar User A2TU781PWGS09X - Historical rating: * title: Citre Shine Moisture Burst Shampoo - 16 fl oz - rating: 2.0 stars
Similar User A3A8F2URN7MEPR - Historical rating: * title: Citre Shine Moisture Burst Shampoo - 16 fl oz - rating: 5.0 stars
Similar User A1U8L4X1O2LXXF - Historical rating: * title: Mehron Makeup Skin Prep Pro (4 oz) - rating: 5.0 stars
Similar User A38IPSLKOQ5O2F - Historical rating: * title: Vktech&reg; 20pcs Mix Colors Rhinestone Nose Studs Ring Bone Bar Pin Piercing Jewelry - rating: 5.0 stars
Constructed Prompt for CF approach:

The prompt:
**********
How will user rate this title: Astra Platinum Double Edge Safety Razor Blades ,100 Blades (20 x 5)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Here are the rating history from users who are similar to this user:
* title

In [17]:

data = pd.read_csv(CF_OUTPUT_PATH)

# Display the original data types
print("Original Data Types:")
print(data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


Original Data Types:
user_id              object
item_id              object
title                object
actual_rating       float64
predicted_rating    float64
dtype: object


Total number of rows with non-float ratings: 0
Rows with non-float ratings:


,user_id,item_id,title,actual_rating,predicted_rating,is_rating_float


In [18]:
# Evaluate CF Model
evaluate_model_predictions_rmse_mae(
    data_path=CF_OUTPUT_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

RMSE: 2.0280 (95% CI: (1.2649, 2.6458)) ± 0.0064
MAE: 1.8049 (95% CI: (0.8000, 2.6000)) ± 0.0087

First few actual vs predicted ratings:
Actual: 5.0, Predicted: 3.0000
Actual: 2.0, Predicted: 5.0000
Actual: 5.0, Predicted: 3.0000
Actual: 5.0, Predicted: 3.0000
Actual: 3.0, Predicted: 3.0000


In [ ]:
# Load the original CF predictions
cf_data = pd.read_csv(CF_OUTPUT_PATH)
cf_data

In [ ]:
def identify_and_rerun_failed_cf_predictions(data, 
                                             pcc_matrix, 
                                             user_column_name='user_id', 
                                             movie_column_name='title', 
                                             movie_id_column='item_id',
                                             rating_column_name='actual_rating', 
                                             num_ratings_per_user=1, 
                                             num_similar_users=4, 
                                             save_path='cf_predictions.csv', 
                                             rerun_save_path='cf_rerun_predictions.csv', 
                                             seed=RANDOM_STATE):
    """
    Identify failed predictions in CF data and rerun them.

    Args:
    - data: DataFrame containing the original CF predictions.
    - pcc_matrix: Pearson Correlation Coefficient matrix.
    - Other arguments for controlling various aspects of the prediction function.

    Returns:
    - Updated DataFrame with rerun predictions.
    """
    # Ensure the original data has the necessary columns
    if rating_column_name not in data.columns:
        raise KeyError(f"Column '{rating_column}' not found in the data.")

    # Identify rows with failed predictions
    failed_rows = data[pd.to_numeric(data['predicted_rating'], errors='coerce').isna()]

    if len(failed_rows) > 0:
        print(f"Re-running predictions for {len(failed_rows)} failed cases.")

        # Call prediction function on failed data
        rerun_data = predict_ratings_with_collaborative_filtering_and_save(
            failed_rows, pcc_matrix,
            user_column_name=user_column_name,
            movie_column_name=movie_column_name,
            movie_id_column=movie_id_column,
            rating_column_name=rating_column_name,
            num_ratings_per_user=num_ratings_per_user,
            num_similar_users=num_similar_users,
            save_path=rerun_save_path,
            seed=seed
        )

        # Update original data with new predictions
        data.loc[failed_rows.index, 'predicted_rating'] = rerun_data['predicted_rating']

    # Save the updated data
    data.to_csv(save_path, index=False)
    print(f"Updated predictions saved to {save_path}")

    return data

# Load the original CF predictions
cf_data = pd.read_csv(CF_OUTPUT_PATH)

# Identify and rerun failed predictions
updated_cf_data = identify_and_rerun_failed_cf_predictions(
    cf_data, pcc_matrix,
    save_path=CF_OUTPUT_PATH,
    rating_column_name='actual_rating', 
    rerun_save_path=CF_RERUN_PATH
)

# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_OUTPUT_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)
